<a href="https://colab.research.google.com/github/Vashi050197/Chatbot/blob/main/LangChain_QA_Panel_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain QA Panel App

In [1]:
!pip install langchain openai chromadb tiktoken pypdf panel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile


In [3]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    main_max_width="690px",
    header_background="#F08080",
)

In [4]:
file_input = pn.widgets.FileInput(width=300)

openaikey = pn.widgets.PasswordInput(
    value="", placeholder="Enter your OpenAI API Key here...", width=300
)
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Run!")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type',
    options=['stuff', 'map_reduce', "refine", "map_rerank"],
    value='map_reduce'
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings"
    ), width=670
)

In [12]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [18]:
import openai

In [19]:
os.environ["OPENAI_API_KEY"] = "sk-***********************unWcsws6g1DNzVdOX"
openai.api_key  = os.environ['OPENAI_API_KEY']
result = qa("/content/new.pdf", "When was GPT-2 created?", "map_reduce", 2)
# result = qa("materials/example.pdf", "When was GPT-2 created?", "map_reduce", 2)

ValueError: ignored

In [11]:
convos = []  # store all panel objects in a list

def qa_result(_):
    os.environ["OPENAI_API_KEY"] = openaikey.value

    # save pdf file to a temp file
    if file_input.value is not None:
        file_input.save("temp.pdf")

        prompt_text = prompt.value
        if prompt_text:
            result = qa(file="temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        "Relevant source text:",
                        pn.pane.Markdown('\n--------------------------------------------------------------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ])
    return pn.Column(*convos, margin=15, width=575, min_height=400)


In [12]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

In [13]:
output = pn.WidgetBox('*Output will show up here:*', qa_interactive, width=670, scroll=True)

In [14]:
# layout
pn.Column(
    pn.pane.Markdown("""
    ## \U0001F60A! Question Answering with your PDF file

    Step 1: Upload a PDF file \n
    Step 2: Enter your OpenAI API key. This costs $$. You will need to set up billing info at [OpenAI](https://platform.openai.com/account). \n
    Step 3: Type your question at the bottom and click "Run" \n

    """),
    pn.Row(file_input,openaikey),
    output,
    widgets

).servable()

Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 467, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 245, in handle_event
 event_cls._handle_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 280, in _handle_event
 cb(event.msg_data)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 390, in trigger_event
 model._trigger_event(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 113, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 260, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 109, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 481, in _comm_event
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 441, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 479, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 416, in _process_bokeh_event
 self._process_event(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/widgets/button.py", line 246, in _process_event
 self.param.trigger('value')
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2437, in trigger
 self_.update(dict(params, **triggers))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2282, in update
 restore = self_._update(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2322, in _update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2506, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2468, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 865, in _replace_pane
 new_object = self.eval(self.object)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 824, in eval
 return eval_function_with_deps(function)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 162, in eval_function_with_deps
 return function(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/depends.py", line 41, in _depends
 return func(*args, **kw)
 File "/usr/local/lib/python3.10/dist-packages/param/reactive.py", line 431, in wrapped
 return eval_fn()(*combined_args, **combined_kwargs)
 File "<ipython-input-11-e5ee3ad41585>", line 12, in qa_result
 result = qa(file="temp.pdf", qu

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] FileInput(width=300)
        [1] PasswordInput(placeholder='Enter your OpenAI A...)
    [2] WidgetBox(css_classes=['panel-widget-box', ...], scroll=True, width=670)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True, sizing_mode='stretch_width')
    [3] Row(width=670)
        [0] Column(margin=5, sizing_mode='stretch_width')
            [0] TextEditor(height=160, placeholder='Enter your questions h..., sizing_mode='stretch_width', toolbar=False)
            [1] Button(name='Run!', sizing_mode='stretch_width')
        [1] Card(sizing_mode='stretch_width', title='Advanced settings')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Column(sizing_mode='stretch_width')
                [0] RadioButtonGroup(name='Chain type', options=['stuff', 'map_reduce', ...], sizing_mode='stretch_width', value='map_reduce')
                [1] IntSlider(end=5, name='Number of relevant c..., sizing_mode='stretch_width', start=1, value=2)